# Strands Agents 세션 간 메모리 유지

이 예제에서는 Strands Agents의 여러 세션에서 메모리를 유지하는 방법을 배웁니다.

`duckduckgo` 검색 API를 사용하여 웹 검색을 수행하는 에이전트의 사용 사례를 사용합니다.

이 노트북에서는 다음을 수행합니다:
- 메모리 기반 Strands 에이전트의 기능 탐색
- 메모리를 저장, 검색 및 나열하는 방법 학습
- 에이전트를 통해 웹 검색을 수행하는 방법 이해
- 대화형 루프에서 에이전트와 상호 작용


### 사용 예시

메모리 저장:
```
커피보다 차를 선호한다는 것을 기억해주세요
```

메모리 검색:
```
내가 마시는 것을 선호하는 것은 무엇인가요?
```

모든 메모리 나열:
```
나에 대해 기억하는 모든 것을 보여주세요
```

### 메모리 사용 팁

- 에이전트에게 정보를 기억하도록 요청할 때 명시적으로 표현하세요
- 관련 메모리를 검색하려면 구체적인 쿼리를 사용하세요
- 메모리 지속성은 더 자연스럽고 맥락적인 대화를 가능하게 합니다

## 설정 및 사전 요구사항

### 사전 요구사항
* Python 3.10+
* AWS 계정 및 환경에 구성된 AWS 자격 증명
* Amazon Bedrock에서 활성화된 Anthropic Claude 3.7
* Amazon Bedrock Knowledge Base, Amazon S3 버킷 및 Amazon DynamoDB를 생성할 수 있는 권한이 있는 IAM 역할

이제 Strands Agent에 필요한 패키지를 설치하겠습니다

In [ ]:
# Install the required packages
!pip install -r requirements.txt

In [ ]:
# Import Required Libraries
import os
from strands import Agent, tool
from strands.models import bedrock
from strands_tools import mem0_memory
import time

from ddgs import DDGS
from ddgs.exceptions import DDGSException, RatelimitException


os.environ["AWS_REGION"] = os.environ.get("AWS_REGION", "us-east-1")
bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0" #Optional: Set a default model for Bedrock

## Mem0 구성

### 메모리 백엔드 옵션

Mem0 Memory Tool은 세 가지 다른 백엔드 구성을 지원합니다:

1. **[OpenSearch](https://aws.amazon.com/opensearch-service/features/serverless/)** (AWS 환경에 권장):
   - AWS 자격 증명 및 OpenSearch 구성 필요
   - `OPENSEARCH_HOST` 설정 및 선택적으로 `AWS_REGION` (기본값 us-west-2)

2. **[FAISS](https://faiss.ai/index.html)** (로컬 개발의 기본값):
   - 로컬 벡터 저장소 백엔드로 FAISS 사용
   - 로컬 벡터 저장을 위해 faiss-cpu 패키지 필요
   - 추가 구성 불필요

3. **Mem0 Platform**:
   - 메모리 관리를 위해 [Mem0 Platform API](https://docs.mem0.ai/platform/quickstart) 사용
   - Mem0 API 키 필요: 환경 변수에 `MEM0_API_KEY` 설정


### 환경 구성

| 환경 변수 | 설명 | 기본값 | 필수 대상 |
|---------------------|-------------|----------|------------|
| OPENSEARCH_HOST | OpenSearch Serverless 호스트 URL | None | OpenSearch |
| AWS_REGION | OpenSearch용 AWS 리전 | us-west-2 | OpenSearch |
| MEM0_API_KEY | Mem0 Platform API 키 | None | Mem0 Platform |
| DEV | 개발 모드 활성화 | false | 모든 모드 |


이 실습의 범위에서는 메모리 관리를 위한 백엔드로 2가지 옵션을 사용할 수 있습니다:
### 옵션 1. [Opensearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/)

AOSS를 위한 설정 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/arch_AOSS.png" width="65%" />
</div>


In [ ]:
# You can manually define your Opensearch Host 
#os.environ["OPENSEARCH_HOST"] = "<your-opensearch-host>.<region>.aoss.amazonaws.com"

In [ ]:
# OR - Run the script to Create Opensearch Serverless resource in your AWS Account
!bash prereqs/deploy_OSS.sh

In [ ]:
wait_time = 120
print(f"Waiting {wait_time} seconds for OpenSearch Serverless collection to be fully operational...")
time.sleep(wait_time)

In [ ]:
# Option 1: Opensearch Serverless
from dotenv import load_dotenv
load_dotenv() # take Opensearch environment variables


### 옵션 2 [Mem0 Platform](https://docs.mem0.ai/platform):

#### [참고]: Opensearch Serverless 옵션을 이미 배포한 경우에는 필요하지 않습니다.

대안으로 [여기](https://docs.mem0.ai/platform/quickstart#2-api-key-setup)의 단계를 따라 Mem0 API 키를 생성하고 환경 변수 **MEM0_API_KEY**로 추가할 수 있습니다.
Mem0 Platform 설정의 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/arch_mem0.png" width="65%" />
</div>


에이전트의 기능을 활성화하려면 AWS 자격 증명 및 OpenSearch / Mem0 Platform에 대한 환경 변수를 구성해야 합니다. 이러한 변수는 메모리 저장 및 검색에 사용됩니다.

In [ ]:
# Option 2: Mem0 API key

#os.environ["MEM0_API_KEY"] = "<your-mem0-api-key>"

## 시스템 프롬프트 정의

`SYSTEM_PROMPT` 변수는 메모리 에이전트의 동작과 기능을 정의합니다. 이 프롬프트는 에이전트가 저장된 메모리를 기반으로 개인화된 응답을 제공하고 필요할 때 웹 검색을 수행하도록 안내합니다.

In [ ]:
# 메모리 작업을 위한 집중형 시스템 프롬프트 정의
SYSTEM_PROMPT = """당신은 사용자를 위한 유용한 개인 비서입니다. 사용자의 기록을 기반으로 맞춤형 응답을 제공하여 사용자를 지원하는 것이 당신의 임무입니다. 

기능:
- mem0_memory 도구(action=“store”)를 사용하여 정보를 저장할 수 있습니다.
- mem0_memory 도구(action=“retrieve”)로 관련 기억을 검색할 수 있습니다.
- duckduckgo_search를 사용해 웹에서 정보를 찾을 수 있습니다.

핵심 규칙:
- 대화체로 자연스럽게 응답하세요.
- 사용자에게 응답하기 전에 항상 기억을 검색하고 이를 바탕으로 답변을 구성하세요.
- 새로운 사용자 정보와 선호도는 mem0_memory에 저장하세요.
- 관련 정보만 공유하십시오.
- 정보가 없을 때는 정중하게 알려주십시오.
"""

## 웹 검색 도구 정의

[Duckduckgo Search API](https://github.com/deedy5/duckduckgo_search)를 사용하는 `websearch` 도구 함수를 통해 에이전트가 웹 검색을 수행할 수 있습니다. 이 함수는 예외를 처리하고 검색 결과 또는 적절한 오류 메시지를 반환합니다.

In [ ]:
@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

## 메모리 에이전트 생성

이제 정의된 도구와 시스템 프롬프트를 사용하여 메모리 중심 에이전트를 초기화합니다. Strands 에이전트는 다음을 수행할 수 있습니다:
1. 컨텍스트를 기반으로 메모리 저장 및 검색. 메모리를 사용하여 더 개인화되고 맥락적인 AI 상호 작용을 생성합니다.
2. DuckDuckGo를 사용하여 웹 검색을 수행하여 최신 정보를 제공합니다.

In [ ]:
# Create an agent with memory, websearch tool
USER_ID = "new_user" # Replace with actual user ID

memory_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[mem0_memory, websearch],
)

## 메모리 작업 시연

다음 예제는 메모리 에이전트를 사용하여 메모리를 저장, 검색 및 나열하는 방법을 보여줍니다.

- **store**: 나중에 검색할 수 있도록 중요한 정보 저장
  - 사용자 선호도 저장
  - 중요한 사실 기억
  - 대화 컨텍스트 유지

- **retrieve**: 쿼리를 기반으로 관련 메모리 액세스
  - 이전에 저장된 정보 찾기
  - 사용자 기록을 기반으로 개인화된 응답 제공

- **list**: 저장된 모든 메모리 보기
  - 어떤 정보가 유지되었는지 확인
  - 저장된 메모리 감사

In [ ]:
# Store initial memories to demonstrate retrieval
memory_agent.tool.mem0_memory(
    action="store", content=f"The user's name is {USER_ID}.", user_id=USER_ID
)
memory_agent.tool.mem0_memory(
    action="store", 
    content="I like to drink tea more than coffee.", 
    user_id=USER_ID
)

In [ ]:
# Retrieve memories
retrieved_memories = memory_agent.tool.mem0_memory(
    action="retrieve", query="What is the user's name?", user_id=USER_ID
)
print("Retrieved Memories:", retrieved_memories)

In [ ]:
# Retrieve memories about preferences
memory_agent.tool.mem0_memory(
    action="retrieve",
    query="What are the my drink preferences?",
    user_id=USER_ID
)

In [ ]:
# Ask the agent a question
response = memory_agent("What are the events happening in the New York today?")
print(response)

In [ ]:
# List all stored memories
print("All Stored Memories:")
all_memories = memory_agent.tool.mem0_memory(
    action="list", user_id=USER_ID
)

## 대화형 에이전트 사용

마지막으로 사용자가 메모리 에이전트와 상호 작용할 수 있는 대화형 루프를 제공합니다. 사용자는 새 메모리를 저장하거나 기존 메모리를 검색하거나 저장된 모든 메모리를 나열할 수 있습니다.

대화형 사용을 테스트하려면:
1. 요구 사항 설치: `pip install -r requirements.txt`
2. Python 파일 `personal_agent_with_memory.py` 실행

## 결론

이 노트북은 Strands 프레임워크를 사용하여 메모리 기능이 있는 개인 에이전트를 만드는 방법을 보여줍니다. 에이전트는 다음을 수행할 수 있습니다:

1. 사용자에 대한 정보 저장
2. 컨텍스트를 기반으로 관련 메모리 검색
3. 추가 정보를 위해 웹 검색
4. 개인화된 응답 제공

이러한 기능을 결합하여 에이전트는 대화 전반에 걸쳐 컨텍스트를 유지하고 시간이 지남에 따라 더 개인화된 지원을 제공할 수 있습니다.

### 정리

이 bash 스크립트를 실행하여 Opensearch Serverless 리소스를 정리합니다. "MEM0_PLATFORM_API"를 사용한 경우에는 실행할 필요가 없습니다.

In [ ]:
!sh prereqs/cleanup_OSS.sh